In [86]:
import torch
import torch.nn as nn
import torch.optim as optim
import scipy.stats as stats
import pandas as pd

# Simulations

## Preliminary Numerical Results

In [87]:
# Define parameters
num_repetitions = 500

rho = 1
p = 1
q = 10
beta_0 = torch.tensor([1])

N_range = [100, 300, 500, 700, 900]
NNp_ratio = 0.1

In [88]:
N = 100

# Generate noises
U = torch.normal(0, 1, size=(N, 1))
V = torch.normal(0, 1, size=(N, p))

# Generate covariates Zj = (E{j+1}+rho*E5)/(1+rho)
Es = torch.rand(N, q)  # uniform [0, 1)
Z = (Es + rho * Es[:, 4].view(-1, 1)) / (1 + rho)


# Define r0(z)
def r0(Z):
    return torch.cos(torch.sum(torch.square(Z), dim=1)).view(-1, 1)


# Generate X
X = r0(Z) + V


# Define f0(x)
def f0(X):
    return X * beta_0


# Define g0(z)
def g0(Z):
    return torch.prod(torch.exp(Z), dim=1).view(-1, 1)


# Generate Y
Y = f0(X) + g0(Z) + U

In [89]:
def train_linear_regression(Z, Y, epochs=1000, lr=0.01):
    """
    Trains a linear regression model using PyTorch for multi-feature inputs (N, q).

    Parameters:
    - Z: Independent variable tensor of shape (N, q)
    - Y: Dependent variable tensor of shape (N, 1)
    - epochs: Number of training iterations
    - lr: Learning rate for optimization

    Returns:
    - trained_model: The trained PyTorch model
    """
    N, q = Z.shape  # Number of samples, number of features

    # Define Linear Regression Model
    class LinearRegression(nn.Module):
        def __init__(self, input_dim):
            super().__init__()
            self.weights = nn.Linear(input_dim, 1, bias=False)  # No intercept

        def forward(self, x):
            return self.weights(x)

    # Initialize model
    model = LinearRegression(input_dim=q)

    # Loss function (MSE)
    criterion = nn.MSELoss()

    # Optimizer (SGD)
    optimizer = optim.SGD(model.parameters(), lr=lr)

    # Training loop
    for epoch in range(epochs):
        optimizer.zero_grad()
        pred_Y = model(Z)
        loss = criterion(pred_Y, Y)
        loss.backward()
        optimizer.step()

    return model  # Return trained model

In [93]:
# Learn g0 and r0 using linear regressions
g0_hat = train_linear_regression(Z, X)
r0_hat = train_linear_regression(torch.cat((X, Z), dim=1), Y)

In [ ]:
def compute_prediction_intervals(model, Z, Y, alpha=0.05):
    """
    Computes prediction intervals for any trained PyTorch model.

    Parameters:
    - model: Trained PyTorch model (supports neural networks)
    - Z: Independent variable tensor (N, q)
    - Y: True dependent variable tensor (N, 1)
    - alpha: Significance level (default: 0.05 for 95% confidence)

    Returns:
    - DataFrame with predicted values and lower/upper prediction intervals
    """
    N, q = Z.shape  # Number of samples, features

    # Get predictions
    pred_Y = model(Z).detach()

    # Compute residuals
    residuals = Y - pred_Y

    # Estimate standard deviation of residuals
    sigma = torch.sqrt((residuals.T @ residuals) / (N - q - 1))  # df = n-q-1

    # Compute Jacobian row-wise (N, q)
    def compute_jacobian_rowwise(model, Z):
        n, q = Z.shape
        jacobian = torch.zeros(n, q)

        for i in range(n):
            Z_i = Z[i].unsqueeze(0).clone().requires_grad_(True)  # Single row
            pred_i = model(Z_i)  # Forward pass for one sample
            grad_outputs = torch.ones_like(pred_i)  # Gradient wrt output

            # Compute gradient wrt Z_i
            grads = torch.autograd.grad(
                outputs=pred_i,
                inputs=Z_i,
                grad_outputs=grad_outputs,
                retain_graph=True,
                create_graph=False,
            )[0]
            jacobian[i] = grads.squeeze()  # Store in jacobian matrix

        return jacobian

    # Compute Jacobian correctly
    jacobian = compute_jacobian_rowwise(model, Z)  # Shape (N, q)

    # Compute standard errors
    se_Y = torch.sqrt(torch.sum(jacobian**2, dim=1, keepdim=True)) * sigma

    # Compute critical t-value for prediction intervals
    t_critical = stats.t.ppf(1 - alpha / 2, df=N - q - 1)

    # Compute 95% prediction intervals
    lower_Y = pred_Y - t_critical * se_Y
    upper_Y = pred_Y + t_critical * se_Y

    # Store results in DataFrame
    df = pd.DataFrame(
        {
            "True_Y": Y.flatten().numpy(),
            "Pred_Y": pred_Y.flatten().numpy(),
            "Lower_Y": lower_Y.flatten().numpy(),
            "Upper_Y": upper_Y.flatten().numpy(),
            "Covered?": ((lower_Y <= Y) & (Y <= upper_Y)).flatten().numpy(),
        }
    )

    return df

In [111]:
compute_prediction_intervals(g0_hat, Z, X)["Covered?"].mean()

np.float64(0.98)